In [13]:
preferences =  [
        [
            {
                "start": 0,
                "startValue": 6.8,
                "end": 1,
                "endValue": 6.8,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 4.5,
                "end": 3,
                "endValue": 4.5,
                "id": 4
            }
        ],
        [
            {
                "start": 0,
                "startValue": 5.4,
                "end": 1,
                "endValue": 5.4,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 2.8,
                "end": 2,
                "endValue": 2.8,
                "id": 4
            },
            {
                "start": 2,
                "startValue": 5.3,
                "end": 3,
                "endValue": 5.3,
                "id": 4
            }
        ],
        [
            {
                "start": 0,
                "startValue": 1.5,
                "end": 1,
                "endValue": 1.5,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 9.5,
                "end": 2,
                "endValue": 9.5,
                "id": 4
            },
            {
                "start": 2,
                "startValue": 3,
                "end": 3,
                "endValue": 3,
                "id": 1
            }
        ]
    ]

In [2]:
preferences_four_agents =  [
        [
            {
                "start": 0,
                "startValue": 6.8,
                "end": 1,
                "endValue": 6.8,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 4.5,
                "end": 3,
                "endValue": 4.5,
                "id": 4
            }
        ],
        [
            {
                "start": 0,
                "startValue": 5.4,
                "end": 1,
                "endValue": 5.4,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 2.8,
                "end": 2,
                "endValue": 2.8,
                "id": 4
            },
            {
                "start": 2,
                "startValue": 5.3,
                "end": 3,
                "endValue": 5.3,
                "id": 4
            }
        ],
        [
            {
                "start": 0,
                "startValue": 6.7,
                "end": 1,
                "endValue": 6.7,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 0.3,
                "end": 2,
                "endValue": 0.3,
                "id": 4
            },
            {
                "start": 2,
                "startValue": 7.9,
                "end": 3,
                "endValue": 7.9,
                "id": 4
            }
        ],
        [
            {
                "start": 0,
                "startValue": 1.5,
                "end": 1,
                "endValue": 1.5,
                "id": 3
            },
            {
                "start": 1,
                "startValue": 9.5,
                "end": 2,
                "endValue": 9.5,
                "id": 4
            },
            {
                "start": 2,
                "startValue": 3,
                "end": 3,
                "endValue": 3,
                "id": 1
            }
        ]
    ]

In [43]:
import numpy as np
import itertools
from scipy.optimize import minimize

In [44]:
cakeSize = 3

In [5]:
'''
Ensures that each agents values the
'''
def normalization(prefs):
    '''
    Ensures that each agents values the
    '''
    #Need to change.
    agents = len(prefs)
    normalization_constants = np.zeros(agents)
    for i in range(agents):
        normalization_constants[i] = value_query_initial(i, prefs, 0, 1)
        for segments in prefs[i]:
            segments['startValue'] /= normalization_constants[i]
            segments['endValue'] /= normalization_constants[i]
    return prefs#, normalizationConstants

In [45]:
def change_bounds(prefs, cakeSize):
    for agents in prefs:
        for segments in agents:
            segments['start'] /= cakeSize
            segments['end'] /= cakeSize
    return prefs

In [18]:
new_prefs =  change_bounds(preferences, cakeSize)

In [46]:
new_prefs_four_agents =  change_bounds(preferences_four_agents, cakeSize)

In [19]:
new_prefs

[[{'start': 0.0,
   'startValue': 6.8,
   'end': 0.3333333333333333,
   'endValue': 6.8,
   'id': 3},
  {'start': 0.3333333333333333,
   'startValue': 4.5,
   'end': 1.0,
   'endValue': 4.5,
   'id': 4}],
 [{'start': 0.0,
   'startValue': 5.4,
   'end': 0.3333333333333333,
   'endValue': 5.4,
   'id': 3},
  {'start': 0.3333333333333333,
   'startValue': 2.8,
   'end': 0.6666666666666666,
   'endValue': 2.8,
   'id': 4},
  {'start': 0.6666666666666666,
   'startValue': 5.3,
   'end': 1.0,
   'endValue': 5.3,
   'id': 4}],
 [{'start': 0.0,
   'startValue': 1.5,
   'end': 0.3333333333333333,
   'endValue': 1.5,
   'id': 3},
  {'start': 0.3333333333333333,
   'startValue': 9.5,
   'end': 0.6666666666666666,
   'endValue': 9.5,
   'id': 4},
  {'start': 0.6666666666666666,
   'startValue': 3,
   'end': 1.0,
   'endValue': 3,
   'id': 1}]]

In [47]:
def find_segments_intervals(prefs):
    breakpoints = set()
    for agents in prefs:
        for segments in agents:
            breakpoints.add(segments['start'])
            breakpoints.add(segments['end'])

    # Convert breakpoints to a sorted list
    sorted_breakpoints = sorted(breakpoints)
    segment_intervals=[]
    for i in range(1, len(sorted_breakpoints)):
        left = sorted_breakpoints[i-1]
        right = sorted_breakpoints[i]
        segment_intervals.append({
                'start': left,
                'end': right
            })
    return segment_intervals

In [48]:
def find_segments(prefs, agents_number):
    segments_intervals = find_segments_intervals(prefs)
    segmented_prefs = [[] for _ in range(agents_number)]
    for i in range(agents_number):
        for segments in prefs[i]:
            for intervals in segments_intervals:
                if (((segments['start'] >= intervals['start']) and (segments['start'] < intervals['end'])) or \
                    ((segments['end'] > intervals['start']) and (segments['end'] <= intervals['end']))):
                    segmented_prefs[i].append({'start': intervals['start'],
                                               'end': intervals['end'],
                                               'value': segments['startValue'],
                                               'area': (intervals['end'] - intervals['start']) * segments['startValue']})
    return segmented_prefs

In [9]:
points = segments_intervals(new_prefs)

NameError: name 'segments_intervals' is not defined

In [10]:
points

NameError: name 'points' is not defined

In [11]:
segments = find_segments(new_prefs, 3)

NameError: name 'new_prefs' is not defined

In [54]:
segments_four_agents = find_segments(new_prefs_four_agents, 4)

In [55]:
segments_four_agents

[[{'start': 0.0,
   'end': 0.3333333333333333,
   'value': 6.8,
   'area': 2.2666666666666666},
  {'start': 0.3333333333333333,
   'end': 0.6666666666666666,
   'value': 4.5,
   'area': 1.5},
  {'start': 0.6666666666666666,
   'end': 1.0,
   'value': 4.5,
   'area': 1.5000000000000002}],
 [{'start': 0.0, 'end': 0.3333333333333333, 'value': 5.4, 'area': 1.8},
  {'start': 0.3333333333333333,
   'end': 0.6666666666666666,
   'value': 2.8,
   'area': 0.9333333333333332},
  {'start': 0.6666666666666666,
   'end': 1.0,
   'value': 5.3,
   'area': 1.7666666666666668}],
 [{'start': 0.0,
   'end': 0.3333333333333333,
   'value': 6.7,
   'area': 2.2333333333333334},
  {'start': 0.3333333333333333,
   'end': 0.6666666666666666,
   'value': 0.3,
   'area': 0.09999999999999999},
  {'start': 0.6666666666666666,
   'end': 1.0,
   'value': 7.9,
   'area': 2.6333333333333337}],
 [{'start': 0.0, 'end': 0.3333333333333333, 'value': 1.5, 'area': 0.5},
  {'start': 0.3333333333333333,
   'end': 0.6666666666

In [13]:
amount_of_segments = len(segments[0])

NameError: name 'segments' is not defined

In [14]:
segments

NameError: name 'segments' is not defined

In [60]:
def solver(segments, agents_number):
    amount_of_segments = len(segments[0])
    cut_positions = []
    assert (agents_number == 3) or (agents_number == 4),\
        "Invalid agents number for algorithm"
    if agents_number == 3:
        for cut_one in range(amount_of_segments):
            for cut_two in range(cut_one, amount_of_segments):
                cut_positions.append([cut_one, cut_two])
    if agents_number == 4:
        for cut_one in range(amount_of_segments):
            for cut_two in range(cut_one, amount_of_segments):
                for cut_three in range(cut_two, amount_of_segments):
                    cut_positions.append([cut_one, cut_two, cut_three])
    agents_list = [i for i in range(agents_number)]
    agents_permutations = list(itertools.permutations(agents_list))
    for agents in agents_permutations:
        for cuts in cut_positions:
            info = find_division(segments, agents, cuts, agents_number)
            if info['envy_free_check'] == True:
                return info['cuts']
            else:
                continue
    return False
    

In [58]:
def find_division(segments, agents, cuts, agents_number):
    if agents_number == 3:
        return find_division_three_agents(segments, agents, cuts)
    if agents_number == 4:
        return find_division_four_agents(segments, agents, cuts)

In [98]:
def constraints_three_agents(segments, agents, cuts, params):
    def constraint1(vars, params):
        cut_one, cut_two = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[2]
        agents = params[3]
        cuts = params[4]
        slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
        slice_two_first_agent_value = middle_slice_value(x, y, segments, agents[0], cuts[0], cuts[1])
        return slice_one_first_agent_value - slice_two_first_agent_value

    def constraint2(vars, params):
        cut_one, cut_two = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[2]
        agents = params[3]
        cuts = params[4]
        slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
        slice_three_first_agent_value = last_slice_value(y, segments, agents[0], cuts[1])
        return slice_one_first_agent_value - slice_three_first_agent_value

    def constraint3(vars, params):
        cut_one, cut_two = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[2]
        agents = params[3]
        cuts = params[4]
        slice_two_second_agent_value = middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
        slice_one_second_agent_value = first_slice_value(x, segments, agents[1], cuts[0])
        return slice_two_second_agent_value - slice_one_second_agent_value

    def constraint4(vars, params):
        cut_one, cut_two = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[2]
        agents = params[3]
        cuts = params[4]
        slice_two_second_agent_value = middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
        slice_three_second_agent_value = last_slice_value(y, segments, agents[1], cuts[1])
        return slice_two_second_agent_value - slice_three_second_agent_value

    def constraint5(vars, params):
        cut_one, cut_two = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[2]
        agents = params[3]
        cuts = params[4]
        slice_three_third_agent_value = last_slice_value(y, segments, agents[2], cuts[1])
        slice_one_third_agent_value = first_slice_value(x, segments, agents[2], cuts[0])
        return slice_three_third_agent_value - slice_one_third_agent_value

    def constraint6(vars, params):
        cut_one, cut_two = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[2]
        agents = params[3]
        cuts = params[4]
        slice_three_third_agent_value = last_slice_value(y, segments, agents[2], cuts[1])
        slice_two_third_agent_value = middle_slice_value(x, y, segments, agents[2], cuts[0], cuts[1])
        return slice_three_third_agent_value - slice_two_third_agent_value

    def constraint7(vars, params):
        cut_one, cut_two = vars
        return cut_two - cut_one

    cons = [{'type': 'ineq', 'fun': constraint1, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint2, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint3, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint4, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint5, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint6, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint7, 'args': (params,)}]

    return cons


In [ ]:
def constraints_four_agents(segments, agents, cuts, params):
    def constraint1(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
        slice_two_first_agent_value = middle_slice_value(x, y, segments, agents[0], cuts[0], cuts[1])
        return slice_one_first_agent_value - slice_two_first_agent_value
            

    def constraint2(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
        slice_three_first_agent_value = middle_slice_value(y, z, segments, agents[0], cuts[1], cuts[2])
        return slice_one_first_agent_value - slice_three_first_agent_value

    def constraint3(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
        slice_four_first_agent_value = last_slice_value(z, segments, agents[0], cuts[2])
        return slice_one_first_agent_value - slice_four_first_agent_value

    def constraint4(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_two_second_agent_value =  middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
        slice_one_second_agent_value = first_slice_value(x, segments, agents[1], cuts[0])
        return slice_two_second_agent_value - slice_one_second_agent_value

    def constraint5(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_two_second_agent_value =  middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
        slice_three_second_agent_value = middle_slice_value(y, z, segments, agents[1], cuts[1], cuts[2])
        return slice_two_second_agent_value - slice_three_second_agent_value

    def constraint6(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_two_second_agent_value =  middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
        slice_four_second_agent_value = last_slice_value(z, segments, agents[1], cuts[2])
        return slice_two_second_agent_value - slice_four_second_agent_value

    def constraint7(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_three_third_agent_value = middle_slice_value(y, z, segments, agents[2], cuts[1], cuts[2])
        slice_one_third_agent_value = first_slice_value(x, segments, agents[2], cuts[0])
        return slice_three_third_agent_value - slice_one_third_agent_value

    def constraint8(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_three_third_agent_value = middle_slice_value(y, z, segments, agents[2], cuts[1], cuts[2])
        slice_two_third_agent_value = middle_slice_value(x, y, segments, agents[2], cuts[0], cuts[1])
        return slice_three_third_agent_value - slice_two_third_agent_value

    def constraint9(vars, params):
        cut_one, cut_two, cut_three = vars
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_three_third_agent_value = middle_slice_value(y, z, segments, agents[2], cuts[1], cuts[2])
        slice_four_third_agent_value = last_slice_value(z, segments, agents[2], cuts[2])
        return slice_three_third_agent_value - slice_four_third_agent_value

    def constraint10(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_four_fourth_agent_value = last_slice_value(z, segments, agents[3], cuts[2])
        slice_one_fourth_agent_value = first_slice_value(x, segments, agents[3], cuts[0])
        return slice_four_fourth_agent_value - slice_one_fourth_agent_value

    def constraint11(vars, params):
        cut_one, cut_two, cut_three = vars
        x = cut_one - params[0]
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_four_fourth_agent_value = last_slice_value(z, segments, agents[3], cuts[2])
        slice_two_fourth_agent_value = middle_slice_value(x, y, segments, agents[3], cuts[0], cuts[1])
        return slice_four_fourth_agent_value - slice_two_fourth_agent_value

    def constraint12(vars, params):
        cut_one, cut_two, cut_three = vars
        y = cut_two - params[1]
        z = cut_three - params[2]
        segments = params[3]
        agents = params[4]
        cuts = params[5]
        slice_four_fourth_agent_value = last_slice_value(z, segments, agents[3], cuts[2])
        slice_three_fourth_agent_value = middle_slice_value(y, z, segments, agents[3], cuts[1], cuts[2])
        return slice_four_fourth_agent_value - slice_three_fourth_agent_value

    def constraint13(vars, params):
        cut_one, cut_two, cut_three = vars
        return cut_two - cut_one

    def constraint14(vars, params):
        cut_one, cut_two, cut_three = vars
        return cut_three - cut_one

    def constraint15(vars, params):
        cut_one, cut_two, cut_three = vars
        return cut_three - cut_two

    cons = [{'type': 'ineq', 'fun': constraint1, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint2, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint3, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint4, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint5, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint6, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint7, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint8, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint9, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint10, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint11, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint12, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint13, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint14, 'args': (params,)},
            {'type': 'ineq', 'fun': constraint15, 'args': (params,)}]

    return cons

In [95]:
def find_division_three_agents(segments, agents, cuts):
    cut_one_lower_bound = segments[0][cuts[0]]['start']
    cut_one_upper_bound = segments[0][cuts[0]]['end']
    cut_two_lower_bound = segments[0][cuts[1]]['start']
    cut_two_upper_bound = segments[0][cuts[1]]['end']

    params = [cut_one_lower_bound, cut_two_lower_bound, segments, agents, cuts]

    # def constraint1(vars, params):
    #     cut_one, cut_two = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[2]
    #     agents = params[3]
    #     cuts = params[4]
    #     slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
    #     slice_two_first_agent_value = middle_slice_value(x, y, segments, agents[0], cuts[0], cuts[1])
    #     return slice_one_first_agent_value - slice_two_first_agent_value

    # def constraint2(vars, params):
    #     cut_one, cut_two = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[2]
    #     agents = params[3]
    #     cuts = params[4]
    #     slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
    #     slice_three_first_agent_value = last_slice_value(y, segments, agents[0], cuts[1])
    #     return slice_one_first_agent_value - slice_three_first_agent_value

    # def constraint3(vars, params):
    #     cut_one, cut_two = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[2]
    #     agents = params[3]
    #     cuts = params[4]
    #     slice_two_second_agent_value = middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
    #     slice_one_second_agent_value = first_slice_value(x, segments, agents[1], cuts[0])
    #     return slice_two_second_agent_value - slice_one_second_agent_value

    # def constraint4(vars, params):
    #     cut_one, cut_two = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[2]
    #     agents = params[3]
    #     cuts = params[4]
    #     slice_two_second_agent_value = middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
    #     slice_three_second_agent_value = last_slice_value(y, segments, agents[1], cuts[1])
    #     return slice_two_second_agent_value - slice_three_second_agent_value

    # def constraint5(vars, params):
    #     cut_one, cut_two = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[2]
    #     agents = params[3]
    #     cuts = params[4]
    #     slice_three_third_agent_value = last_slice_value(y, segments, agents[2], cuts[1])
    #     slice_one_third_agent_value = first_slice_value(x, segments, agents[2], cuts[0])
    #     return slice_three_third_agent_value - slice_one_third_agent_value

    # def constraint6(vars, params):
    #     cut_one, cut_two = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[2]
    #     agents = params[3]
    #     cuts = params[4]
    #     slice_three_third_agent_value = last_slice_value(y, segments, agents[2], cuts[1])
    #     slice_two_third_agent_value = middle_slice_value(x, y, segments, agents[2], cuts[0], cuts[1])
    #     return slice_three_third_agent_value - slice_two_third_agent_value

    # def constraint7(vars, params):
    #     cut_one, cut_two = vars
    #     return cut_two - cut_one

    # cons = [{'type': 'ineq', 'fun': constraint1, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint2, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint3, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint4, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint5, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint6, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint7, 'args': (params,)}]

    cons = constraints_three_agents(segments, agents, cuts, params)

    initial_guess = [cut_one_lower_bound, cut_two_lower_bound]

    cut_bounds = [(cut_one_lower_bound, cut_one_upper_bound), (cut_two_lower_bound, cut_two_upper_bound)]

    def objective(vars, *arge):
        return 0

    result = minimize(objective, initial_guess, args=(params,), constraints=cons, bounds= cut_bounds)

    if result.success:
        return {'envy_free_check':True, 'cuts': result.x}
    else:
        return {'envy_free_check':False, 'cuts': None}

In [96]:
def find_division_four_agents(segments, agents, cuts):
    cut_one_lower_bound = segments[0][cuts[0]]['start']
    cut_one_upper_bound = segments[0][cuts[0]]['end']
    cut_two_lower_bound = segments[0][cuts[1]]['start']
    cut_two_upper_bound = segments[0][cuts[1]]['end']
    cut_three_lower_bound = segments[0][cuts[2]]['start']
    cut_three_upper_bound = segments[0][cuts[2]]['end']
    params = [cut_one_lower_bound, cut_two_lower_bound, cut_three_lower_bound, segments, agents, cuts]

    # def constraint1(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
    #     slice_two_first_agent_value = middle_slice_value(x, y, segments, agents[0], cuts[0], cuts[1])
    #     return slice_one_first_agent_value - slice_two_first_agent_value
            

    # def constraint2(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
    #     slice_three_first_agent_value = middle_slice_value(y, z, segments, agents[0], cuts[1], cuts[2])
    #     return slice_one_first_agent_value - slice_three_first_agent_value

    # def constraint3(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_one_first_agent_value = first_slice_value(x, segments, agents[0], cuts[0])
    #     slice_four_first_agent_value = last_slice_value(z, segments, agents[0], cuts[2])
    #     return slice_one_first_agent_value - slice_four_first_agent_value

    # def constraint4(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_two_second_agent_value =  middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
    #     slice_one_second_agent_value = first_slice_value(x, segments, agents[1], cuts[0])
    #     return slice_two_second_agent_value - slice_one_second_agent_value

    # def constraint5(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_two_second_agent_value =  middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
    #     slice_three_second_agent_value = middle_slice_value(y, z, segments, agents[1], cuts[1], cuts[2])
    #     return slice_two_second_agent_value - slice_three_second_agent_value

    # def constraint6(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_two_second_agent_value =  middle_slice_value(x, y, segments, agents[1], cuts[0], cuts[1])
    #     slice_four_second_agent_value = last_slice_value(z, segments, agents[1], cuts[2])
    #     return slice_two_second_agent_value - slice_four_second_agent_value

    # def constraint7(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_three_third_agent_value = middle_slice_value(y, z, segments, agents[2], cuts[1], cuts[2])
    #     slice_one_third_agent_value = first_slice_value(x, segments, agents[2], cuts[0])
    #     return slice_three_third_agent_value - slice_one_third_agent_value

    # def constraint8(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_three_third_agent_value = middle_slice_value(y, z, segments, agents[2], cuts[1], cuts[2])
    #     slice_two_third_agent_value = middle_slice_value(x, y, segments, agents[2], cuts[0], cuts[1])
    #     return slice_three_third_agent_value - slice_two_third_agent_value

    # def constraint9(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_three_third_agent_value = middle_slice_value(y, z, segments, agents[2], cuts[1], cuts[2])
    #     slice_four_third_agent_value = last_slice_value(z, segments, agents[2], cuts[2])
    #     return slice_three_third_agent_value - slice_four_third_agent_value

    # def constraint10(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_four_fourth_agent_value = last_slice_value(z, segments, agents[3], cuts[2])
    #     slice_one_fourth_agent_value = first_slice_value(x, segments, agents[3], cuts[0])
    #     return slice_four_fourth_agent_value - slice_one_fourth_agent_value

    # def constraint11(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     x = cut_one - params[0]
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_four_fourth_agent_value = last_slice_value(z, segments, agents[3], cuts[2])
    #     slice_two_fourth_agent_value = middle_slice_value(x, y, segments, agents[3], cuts[0], cuts[1])
    #     return slice_four_fourth_agent_value - slice_two_fourth_agent_value

    # def constraint12(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     y = cut_two - params[1]
    #     z = cut_three - params[2]
    #     segments = params[3]
    #     agents = params[4]
    #     cuts = params[5]
    #     slice_four_fourth_agent_value = last_slice_value(z, segments, agents[3], cuts[2])
    #     slice_three_fourth_agent_value = middle_slice_value(y, z, segments, agents[3], cuts[1], cuts[2])
    #     return slice_four_fourth_agent_value - slice_three_fourth_agent_value

    # def constraint13(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     return cut_two - cut_one

    # def constraint14(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     return cut_three - cut_one

    # def constraint15(vars, params):
    #     cut_one, cut_two, cut_three = vars
    #     return cut_three - cut_two

    # cons = [{'type': 'ineq', 'fun': constraint1, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint2, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint3, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint4, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint5, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint6, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint7, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint8, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint9, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint10, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint11, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint12, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint13, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint14, 'args': (params,)},
    #         {'type': 'ineq', 'fun': constraint15, 'args': (params,)}]

    cons = constraints_four_agents(segments, agents, cuts, params)

    initial_guess = [cut_one_lower_bound, cut_two_lower_bound, cut_three_lower_bound]

    cut_bounds = [(cut_one_lower_bound, cut_one_upper_bound), (cut_two_lower_bound, cut_two_upper_bound),
                  (cut_three_lower_bound, cut_three_upper_bound)]

    def objective(vars, *arge):
        return 0

    result = minimize(objective, initial_guess, args=(params,), constraints=cons, bounds= cut_bounds)

    if result.success:
        return {'envy_free_check':True, 'cuts': result.x}
    else:
        return {'envy_free_check':False, 'cuts': None}

In [17]:
vars = (1,2)
contraints1(vars, params)

NameError: name 'contraints1' is not defined

In [18]:
check = make_first_slice_value_function(segments, 0, 1)

NameError: name 'make_first_slice_value_function' is not defined

In [19]:
check(0)

NameError: name 'check' is not defined

In [20]:
check(1/3)

NameError: name 'check' is not defined

In [21]:
cuts = solver(segments, 3)

NameError: name 'segments' is not defined

In [97]:
cuts = solver(segments_four_agents, 4)

NameError: name 'params' is not defined

In [71]:
middle_slice_value(0.333333, 0.333333, segments_four_agents, 0, 0, 0)

0.0

In [73]:
cuts

array([0.21339025, 0.47454238, 0.78258352])

In [74]:
lower_one = 1/3

In [75]:
lower_two = 2/3

In [76]:
first_slice_value(cuts[0], segments_four_agents, 0, 0)

1.4510536823714628

In [77]:
middle_slice_value(cuts[0], cuts[1] - lower_one, segments_four_agents, 0, 0, 1)

1.4510536874590838

In [78]:
middle_slice_value(cuts[1] - lower_one, cuts[2] - lower_two, segments_four_agents, 0, 1, 2)

1.3861851428412297

In [79]:
last_slice_value(cuts[2] - lower_two, segments_four_agents, 0, 2)

0.9783741539948902

In [80]:
first_slice_value(cuts[0], segments_four_agents, 1, 0)

1.1523073360008675

In [81]:
middle_slice_value(cuts[0], cuts[1] - lower_one, segments_four_agents, 1, 0, 1)

1.0430779904122134

In [82]:
middle_slice_value(cuts[1] - lower_one, cuts[2] - lower_two, segments_four_agents, 1, 1, 2)

1.152307336659604

In [83]:
last_slice_value(cuts[2] - lower_two, segments_four_agents, 1, 2)

1.1523073369273151

In [84]:
first_slice_value(cuts[0], segments_four_agents, 2, 0)

1.429714657630706

In [85]:
middle_slice_value(cuts[0], cuts[1] - lower_one, segments_four_agents, 2, 0, 1)

0.8459813892468858

In [86]:
middle_slice_value(cuts[1] - lower_one, cuts[2] - lower_two, segments_four_agents, 2, 1, 2)

0.9733804383313789

In [87]:
last_slice_value(cuts[2] - lower_two, segments_four_agents, 2, 2)

1.7175901814576962

In [88]:
first_slice_value(cuts[0], segments_four_agents, 3, 0)

0.3200853711113521

In [89]:
middle_slice_value(cuts[0], cuts[1] - lower_one, segments_four_agents, 3, 0, 1)

1.5214005577901726

In [90]:
middle_slice_value(cuts[1] - lower_one, cuts[2] - lower_two, segments_four_agents, 3, 1, 2)

2.1729313017685485

In [91]:
last_slice_value(cuts[2] - lower_two, segments_four_agents, 3, 2)

0.6522494359965935

In [23]:
segments[0]

NameError: name 'segments' is not defined

In [24]:
segments[1]

NameError: name 'segments' is not defined

In [25]:
segments[2]

NameError: name 'segments' is not defined

In [26]:
lower = segments[0][1]['start']

NameError: name 'segments' is not defined

In [27]:
first_slice_value(cuts[0], segments, 0, 0)

NameError: name 'first_slice_value' is not defined

In [28]:
middle_slice_value(cuts[0], cuts[1] - lower, segments, 0, 0, 1)

NameError: name 'middle_slice_value' is not defined

In [29]:
last_slice_value(cuts[1] - lower, segments, 0, 1)

NameError: name 'last_slice_value' is not defined

In [30]:
first_slice_value(cuts[0], segments, 1, 0)

NameError: name 'first_slice_value' is not defined

In [31]:
middle_slice_value(cuts[0], cuts[1] - lower, segments, 1, 0, 1)

NameError: name 'middle_slice_value' is not defined

In [32]:
last_slice_value(cuts[1] - lower, segments, 1, 1)

NameError: name 'last_slice_value' is not defined

In [33]:
first_slice_value(cuts[0], segments, 2, 0)

NameError: name 'first_slice_value' is not defined

In [34]:
middle_slice_value(cuts[0], cuts[1] - lower, segments, 2, 0, 1)

NameError: name 'middle_slice_value' is not defined

In [35]:
last_slice_value(cuts[1] - lower, segments, 2, 1)

NameError: name 'last_slice_value' is not defined

In [36]:
if middle_slice_value(cuts[0], cuts[1] - 1/3, segments, 2, 0, 1) == last_slice_value(cuts[1] - 1/3, segments, 2, 1):
    print("yes")

NameError: name 'middle_slice_value' is not defined

In [51]:
def first_slice_value(x, segments, agent, cut):
    first_agent_constant_value = 0
    cut_range = segments[agent][cut]['end'] - segments[agent][cut]['start']
    for i in range(cut):
        first_agent_constant_value += segments[agent][i]['area']
    first_agent_variable_value = segments[agent][cut]['value']
    if ((0 <= x) and (x <= cut_range)):
        value = first_agent_constant_value + x * first_agent_variable_value 
        return value 
    else:
        return "Input out of bounds"

In [70]:
def middle_slice_value(x, y, segments, agent, cut_one, cut_two):
    cut_one_range = segments[agent][cut_one]['end'] - segments[agent][cut_one]['start']
    cut_two_range = segments[agent][cut_two]['end'] - segments[agent][cut_two]['start']
    middle_agent_constant_value = 0
    for i in range(cut_one+1, cut_two):
        middle_agent_constant_value += segments[agent][i]['area']
    middle_agent_variable_value_component_one = segments[agent][cut_one]['value']
    middle_agent_variable_value_component_two = segments[agent][cut_two]['value']
    assert (((0 <= x) and (x <= cut_one_range)) and ((0 <= y) and (y <= cut_two_range))), \
        "cut out of range"
    if cut_one != cut_two:
        value = middle_agent_constant_value + (cut_one_range - x) * middle_agent_variable_value_component_one + \
                y * middle_agent_variable_value_component_two
    else:
        value = middle_agent_constant_value + (y - x) * middle_agent_variable_value_component_one
    return value

In [53]:
def last_slice_value(y, segments, agent, cut):
    last_agent_constant_value = 0
    cut_range = segments[agent][cut]['end'] - segments[agent][cut]['start']
    amount_of_segments = len(segments[agent])
    for i in range(cut+1, amount_of_segments):
        last_agent_constant_value += segments[agent][i]['area']
    last_agent_variable_value = segments[agent][cut]['value']
    if ((0 <= y) and (y <= cut_range)):
        value =  last_agent_constant_value + (cut_range - y) * last_agent_variable_value
        return value
    else:
        return "Input out of bounds"

In [40]:
solver(segments, 3)

NameError: name 'segments' is not defined

In [41]:
for i in range(0):
    print(i)